# Kinematic decoding

This notebook compares the decoding performance of MARBLE with CEBRA and TDR on a macaque centre-out reaching task. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

!pip install statannotations ipympl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from statannotations.Annotator import Annotator
from sklearn.model_selection import KFold
from macaque_reaching_helpers import *
from tqdm import tqdm

Load firing rate and kinematics data

In [ ]:
!mkdir data
!wget -nc https://dataverse.harvard.edu/api/access/datafile/6969885 -O data/kinematics.pkl

with open('data/kinematics.pkl', 'rb') as handle:
    data = pickle.load(handle)

# Load MARBLE and CEBRA embeddings

In [ ]:
!wget -nc https://dataverse.harvard.edu/api/access/datafile/7062022 -O data/marble_embeddings_out20_pca5_100ms.pkl
!wget -nc https://dataverse.harvard.edu/api/access/datafile/7509031 -O data/cebra_embeddings_out20_pca5_100ms.pkl

with open('data/marble_embeddings_out20_pca5_100ms.pkl', 'rb') as handle:
    _, marble_embeddings, _, _, trial_ids, _  = pickle.load(handle)

with open('data/cebra_embeddings_out20_pca5_100ms.pkl', 'rb') as handle:
    _, cebra_embeddings, _, _, _, _ = pickle.load(handle)

# Load raw firing rates

In [ ]:
pca_n = 5
filter_data = True
conditions=['DownLeft','Left','UpLeft','Up','UpRight','Right','DownRight']  

!wget -nc https://dataverse.harvard.edu/api/access/datafile/6969883 -O data/rate_data_20ms_100ms.pkl

with open('data/rate_data_20ms_100ms.pkl', 'rb') as handle:
    rates = pickle.load(handle)

!wget -nc https://dataverse.harvard.edu/api/access/datafile/6963200 -O data/trial_ids.pkl

with open('data/trial_ids.pkl', 'rb') as handle:
    trials = pickle.load(handle)
    
pos, pos_raw = [], []
for day in rates.keys():
    #preprocess by PCA dimensionality reduction and smoothing
    pca = fit_pca(rates, day, conditions, filter_data=filter_data, pca_n=pca_n)
    pos_, _, _, _, _ = format_data(rates, 
                                   trials,
                                   day, 
                                   conditions, 
                                   pca=pca,
                                   filter_data=filter_data,
                                  )

    #no preprocessing for comparison
    pos_raw_, _, _, _, _ = format_data(rates, 
                                   trials,
                                   day, 
                                   conditions,
                                  )
    
    pos.append(np.vstack(pos_))
    pos_raw.append(np.vstack(pos_raw_))

### Targeted Dimensionality Reduction (TDR)

In [ ]:
condition_labels = {'DownLeft': [1,[-1, -1]], 
                    'Left': [2, [-1, 0]], 
                    'UpLeft': [3,[-1, 1]], 
                    'Up': [4,[0, 1]], 
                    'UpRight': [5,[1, 1]], 
                    'Right': [6,[1, 0]], 
                    'DownRight': [7,[1, -1]]} 

TDR_embeddings = {}
for day in tqdm(rates.keys()):
    unique_trial_ids = np.unique(trial_ids[day])
    Z, X, cond = [], [], []
    for t in unique_trial_ids:
        c_l = data[day][t]['condition']
        firing_rates = pos_raw[day][trial_ids[day]==t,:].T
        c = np.tile(condition_labels[c_l][1], (firing_rates.shape[1],1))
        regressors = np.hstack([c, np.ones([firing_rates.shape[1], 1])])
        
        Z.append(firing_rates)
        X.append(regressors)
        cond.append(condition_labels[c_l][0])
    
    Z = np.stack(Z, axis=2)
    X = np.stack(X, axis=2)
    cond = np.hstack(cond)
    
    #standardise per neuron
    Z -= Z.mean(axis=(1,2), keepdims=True)
    Z /= Z.std(axis=(1,2), keepdims=True)
    
    [n, T, tr] = Z.shape
    n_reg = X.shape[1]
    
    #compute TDR regression coefficients
    betaBehav2Neural = np.zeros([T,n,n_reg-1])
    for i in range(T):
        Ztrain = Z[:,i,:].T
        Xtrain = X[i,:,:].T
    
        reg = np.linalg.lstsq(Xtrain, Ztrain, rcond=None)[0]
        reg = np.linalg.pinv(reg) # Compute the TDR axes.
        reg = reg[:,:-1] # remove last regressor (bias)
    
        betaBehav2Neural[i,:,:] = reg
    
    #project data to TDR subspace
    Zproj = np.zeros([n_reg-1,T,tr])
    for i in range(T):
        Zt = Z[:,i,:].T 
        regt = betaBehav2Neural[i,:,:]
        Zproj[:,i,:] = (Zt @ regt).T

    TDR_embeddings[day] = Zproj

In [ ]:
#plot
plt.figure()
colors = plt.cm.viridis(np.linspace(0,1,7))
for j in range(tr):
    c = cond[j]-1
    plt.plot(Zproj[0,:,j], Zproj[1,:,j], c = colors[c])

In [ ]:
# match the neural representations to the kinematics
for day in data.keys():
    unique_trial_ids = np.unique(trial_ids[day])
    for i, t in enumerate(unique_trial_ids):
        data[day][t]['kinematics'] = data[day][t]['kinematics'][:,:-1] #remove last point because
        data[day][t]['lfads_factors'] = data[day][t]['lfads_factors'][:,:-1] 
        data[day][t]['marble_emb'] = marble_embeddings[day][trial_ids[day]==t,:].T
        data[day][t]['firing_rates'] = pos[day][trial_ids[day]==t,:].T
        data[day][t]['cebra_emb'] = cebra_embeddings[day][trial_ids[day]==t,:].T
        data[day][t]['raw_firing_rates'] = pos_raw[day][trial_ids[day]==t,:].T
        data[day][t]['TDR_emb'] = TDR_embeddings[day][:,:,i]

# Visualise kinematics for a single session

In [ ]:
session = 0

colors = plt.cm.viridis(np.linspace(0,1,7))

# plot average kinematic position across trials for a given session
plt.figure()
for c,cond in enumerate(conditions):   
    meh = np.dstack([data[session][t]['kinematics'] for t in data[session].keys() if data[session][t]['condition']==cond]).mean(2)        
    plt.plot(meh[0,:],meh[1,:],c=colors[c])
plt.title('average kinematic hand position across trials')

# plot kinematic position for each trials in a given session
plt.figure()
for c,cond in enumerate(conditions):   
    for t in data[session].keys():
        if data[session][t]['condition']==cond:
            meh = data[session][t]['kinematics']
            plt.plot(meh[0,:],meh[1,:],c=colors[c])
plt.title('per trial kinematic hand position')

# Decoding single session

### Optimal linear decoding via LFADS, MARBLE and CEBRA

In [ ]:
session = 0

unique_trial_ids = np.unique(trial_ids[session])

L_lfads = train_OLE(data[session], trial_ids[session], representation='lfads_factors')

# loop over test trials
for tr in unique_trial_ids:
    trial_pred = decode_kinematics(data[session][tr], L_lfads, dt=20, representation='lfads_factors')
    data[session][tr]['lfads_decoded'] = trial_pred

L_firing_rates = train_OLE(data[session], trial_ids[session], representation='firing_rates')

# loop over test trials
for tr in unique_trial_ids:
    trial_pred = decode_kinematics(data[session][tr], L_firing_rates, dt=20, representation='firing_rates')
    data[session][tr]['firing_rates_decoded'] = trial_pred

L_marble = train_OLE(data[session], trial_ids[session], representation='marble_emb')

# loop over test trials
for tr in unique_trial_ids:
    trial_pred = decode_kinematics(data[session][tr], L_marble, dt=20, representation='marble_emb')
    data[session][tr]['marble_decoded'] = trial_pred
    
L_cebra = train_OLE(data[session], trial_ids[session], representation='cebra_emb')

# loop over test trials
for tr in unique_trial_ids:
    trial_pred = decode_kinematics(data[session][tr], L_cebra, dt=20, representation='cebra_emb')
    data[session][tr]['cebra_decoded'] = trial_pred

L_TDR = train_OLE(data[session], trial_ids[session], representation='TDR_emb')

# loop over test trials
for tr in unique_trial_ids:
    trial_pred = decode_kinematics(data[session][tr], L_TDR, dt=20, representation='TDR_emb')
    data[session][tr]['TDR_decoded'] = trial_pred

### Comparison of decoding with ground truth

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(10,4))

plot_kinematics(data, session, unique_trial_ids, representation='kinematics', ax=ax[0,0])
plot_kinematics(data, session, unique_trial_ids, representation='firing_rates_decoded', ax=ax[0,1])
plot_kinematics(data, session, unique_trial_ids, representation='marble_decoded', ax=ax[0,2])
plot_kinematics(data, session, unique_trial_ids, representation='lfads_decoded', ax=ax[1,0])
plot_kinematics(data, session, unique_trial_ids, representation='cebra_decoded', ax=ax[1,1])
plot_kinematics(data, session, unique_trial_ids, representation='TDR_decoded', ax=ax[1,2])

# Decode across all sessions

Above we decoded for a single session. Lets now loop over every session and compute some quantitative comparisons with the ground truth kinematics.

In [ ]:
kf = KFold(n_splits=5, shuffle=True) # use 5-fold split of the data 

r2_lfads_vel = []; r2_lfads_pos = []
r2_cebra_vel = []; r2_cebra_pos = []
r2_marble_vel = []; r2_marble_pos = []
r2_TDR_vel = []; r2_TDR_pos = []
r2_firing_rates_vel = []; r2_firing_rates_pos = []

# loop over seessions
for d in tqdm(data.keys()):
    unique_trial_ids = np.unique(trial_ids[d])
    
    # cross validation
    for i, (train_index, test_index) in enumerate(kf.split(unique_trial_ids)):

        train_data = {key: data[d][key] for key in train_index if key in data[d]}

        #LFADS
        Lw = train_OLE(data[d], unique_trial_ids[train_index], representation='lfads_factors')
        
        for tr in unique_trial_ids[test_index]:
            trial_pred = decode_kinematics(data[d][tr], Lw, dt=20, representation='lfads_factors')
            data[d][tr]['lfads_decoded'] = trial_pred
           
        #CEBRA
        Lw = train_OLE(data[d], unique_trial_ids[train_index], representation='cebra_emb')
        
        for tr in unique_trial_ids[test_index]:
            trial_pred = decode_kinematics(data[d][tr], Lw, dt=20, representation='cebra_emb')
            data[d][tr]['cebra_decoded'] = trial_pred
            
        #MARBLE
        Lw = train_OLE(data[d], unique_trial_ids[train_index], representation='marble_emb')
        
        for tr in unique_trial_ids[test_index]:
            trial_pred = decode_kinematics(data[d][tr], Lw, dt=20, representation='marble_emb')
            data[d][tr]['marble_decoded'] = trial_pred

        #TDR
        Lw = train_OLE(data[d], unique_trial_ids[train_index], representation='TDR_emb')
        
        for tr in unique_trial_ids[test_index]:
            trial_pred = decode_kinematics(data[d][tr], Lw, dt=20, representation='TDR_emb')
            data[d][tr]['TDR_decoded'] = trial_pred

        #Firing rates
        Lw = train_OLE(data[d], unique_trial_ids[train_index], representation='firing_rates')
        
        for tr in unique_trial_ids[test_index]:
            trial_pred = decode_kinematics(data[d][tr], Lw, dt=20, representation='firing_rates')
            data[d][tr]['firing_rates_decoded'] = trial_pred
            
    # r-squared velocity
    r2_pos, r2_vel = correlation(data[d], unique_trial_ids, representation='lfads_decoded')   
    r2_lfads_pos.append(r2_pos)
    r2_lfads_vel.append(r2_vel)
    
    r2_pos, r2_vel = correlation(data[d], unique_trial_ids, representation='cebra_decoded')   
    r2_cebra_pos.append(r2_pos)
    r2_cebra_vel.append(r2_vel)
    
    r2_pos, r2_vel = correlation(data[d], unique_trial_ids, representation='marble_decoded')   
    r2_marble_pos.append(r2_pos)
    r2_marble_vel.append(r2_vel)

    r2_pos, r2_vel = correlation(data[d], unique_trial_ids, representation='TDR_decoded')   
    r2_TDR_pos.append(r2_pos)
    r2_TDR_vel.append(r2_vel)

    r2_pos, r2_vel = correlation(data[d], unique_trial_ids, representation='firing_rates_decoded')   
    r2_firing_rates_pos.append(r2_pos)
    r2_firing_rates_vel.append(r2_vel)

Lets now visualise the decoded kinematics for the same set of example sessions (Fig S7).

In [ ]:
# chosen example sessions
examples = [5,6,8,11,14,15,18,23,26,32]

fig, ax = plt.subplots(4,len(examples),figsize=(15,5))

for i,d in enumerate(examples):
    
    unique_trial_ids = np.unique(trial_ids[d])

    ax[0,i] = plot_kinematics(data, d, unique_trial_ids, representation='kinematics', ax=ax[0,i])
    ax[1,i] = plot_kinematics(data, d, unique_trial_ids, representation='marble_decoded', ax=ax[1,i])
    ax[2,i] = plot_kinematics(data, d, unique_trial_ids, representation='cebra_decoded', ax=ax[2,i])
    ax[3,i] = plot_kinematics(data, d, unique_trial_ids, representation='lfads_decoded', ax=ax[3,i])
    ax[3,i] = plot_kinematics(data, d, unique_trial_ids, representation='firing_rates_decoded', ax=ax[3,i])

## Instantaneous velocity decoding

In [ ]:
results = pd.DataFrame(data=np.vstack([ r2_marble_vel, r2_cebra_vel, r2_lfads_vel, r2_TDR_vel, r2_firing_rates_vel]).T,columns=['marble', 'CEBRA', 'LFADS', 'TDR', 'firing_rates'])
results = results.melt()
results.columns = ['model','accuracy']

f, ax = plt.subplots(figsize=(4,5))
sns.despine(bottom=True, left=True)

sns.stripplot(
    data=results, x="model", y="accuracy",
    dodge=True, alpha=.5, zorder=1,
)

sns.pointplot(
    data=results, x="model", y="accuracy",
    join=False, dodge=.8 - .8 / 3, palette="dark",
    markers="d", scale=.75, errorbar=None
)

pairs=[("LFADS", "marble"), ("CEBRA","marble"), ("firing_rates","marble"), ("TDR","marble")]

annotator = Annotator(ax, pairs, data=results, x="model", y="accuracy",)
annotator.configure(test='Wilcoxon', text_format='star', loc='outside')
annotator.apply_and_annotate()

## Decoding final reach direction

In [ ]:
marble_model_acc = []
cebra_model_acc = []
lfads_model_acc = []
TDR_model_acc = []
firing_rates_model_acc = []

for d in tqdm(data.keys()):           

    unique_trial_ids = np.unique(trial_ids[d])
    
    # fit classifier to kinematics
    clf = fit_classifier(data[d], conditions, unique_trial_ids, representation='kinematics')
    
    # evaluate classifier on marble decoded
    score = transform_classifier(clf, data[d], conditions, unique_trial_ids, representation='marble_decoded')
    marble_model_acc.append(score)
    
    # evaluate classifier on cebra decoded
    score = transform_classifier(clf, data[d], conditions, unique_trial_ids, representation='cebra_decoded')
    cebra_model_acc.append(score)
    
    # evaluate classifier on lfads decoded
    score = transform_classifier(clf, data[d], conditions, unique_trial_ids, representation='lfads_decoded')
    lfads_model_acc.append(score)

    # evaluate classifier on lfads decoded
    score = transform_classifier(clf, data[d], conditions, unique_trial_ids, representation='TDR_decoded')
    TDR_model_acc.append(score)

    # evaluate classifier on firing_rates
    score = transform_classifier(clf, data[d], conditions, unique_trial_ids, representation='firing_rates_decoded')
    firing_rates_model_acc.append(score)

results = pd.DataFrame(data=np.vstack([ marble_model_acc, cebra_model_acc, lfads_model_acc, TDR_model_acc, firing_rates_model_acc]).T,columns=['marble', 'cebra', 'LFADS', 'TDR', 'firing_rates'])

results = results.melt()
results.columns = ['model','accuracy']

In [ ]:
f, ax = plt.subplots(figsize=(4,4))
sns.despine(bottom=True, left=True)

sns.stripplot(
    data=results, x="model", y="accuracy",
    dodge=True, alpha=.5, zorder=1,
)

sns.pointplot(
    data=results, x="model", y="accuracy",
    join=False, dodge=.8 - .8 / 3, palette="dark",
    markers="d", scale=.75, errorbar=None
)